**references**

https://www.statworx.com/at/blog/how-to-build-a-dashboard-in-python-plotly-dash-step-by-step-tutorial/
https://github.com/STATWORX/blog/blob/master/DashApp/app_basic.py

https://pierpaolo28.github.io/blog/blog21/
https://github.com/pierpaolo28/Data-Visualization/tree/master/Dash

https://dash.plotly.com/deployment

# https://plotly.com/python/filled-area-plots/

between area: same color, but no different colors: color btw PMA and MA

In [1]:
!pip install dash

In [2]:
!pip install yfinance --upgrade --no-cache-dir

In [3]:
#!pip install dash

import dash ## local 
#from jupyter_dash import JupyterDash # colab
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

from sklearn import preprocessing
import numpy as np
import tensorflow as tf

import yfinance as yf

<ipython-input-3-7e2eed0b809f>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-3-7e2eed0b809f>:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [4]:
stock_list = ["FB", "AAPL", "AMZN", "NFLX", "GOOG", "MSFT", "XLK", "QQQ"]

In [5]:
# to get predict data
stock_dic = dict()

for stock in stock_list:
    df = yf.download(stock, start="2019-7-25").drop(columns='Adj Close')
    df.dropna(inplace=True)
    stock_dic[stock] = df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# predict1, predict2, predict3, MA, PMA, UpDown

In [6]:
# get normalizer
df1_dic={}
for stock in stock_list:
    data = yf.download(stock, start="2012-05-18", end="2020-07-23")
    # data, meta_data = ts.get_daily(symbol=stock,outputsize = 'full')
    # data.sort_values(by='date', inplace=True) 
    data = data.drop(columns='Adj Close')
    df1_dic[stock] = data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
history_points = 50
def getXtest(data, data_nor):
    data = data.to_numpy()
    data2 = data_nor.to_numpy()
    
    data_normaliser = preprocessing.MinMaxScaler()
    data_normaliser = data_normaliser.fit(data)
    data_normalised = data_normaliser.transform(data)
    
    Xtest = np.array([data_normalised[i : i + history_points].copy() for i in range(len(data) - history_points+1)])
#     next_day_open_values = np.array([data2[:,0][i + history_points].copy() for i in range(len(data2) - history_points)])
    next_day_open_values = np.array([data[:,0][i + history_points].copy() for i in range(len(data) - history_points)])
    next_day_open_values = np.expand_dims(next_day_open_values, -1)
  
    next_day_close_values = np.array([data[:,3][i + history_points].copy() for i in range(len(data) - history_points)])
    next_day_close_values = np.expand_dims(next_day_close_values, -1)
    
    y_normaliser_open = preprocessing.MinMaxScaler()
    y_normaliser_open.fit(next_day_open_values)
    
    y_normaliser_close = preprocessing.MinMaxScaler()
    y_normaliser_close.fit(next_day_close_values)

    return Xtest, y_normaliser_open, y_normaliser_close

In [8]:
# 1 day prediction: close, open (for candle stick)
def getResult1(stock):
    data = stock_dic[stock]
    data_nor = df1_dic[stock]
    
    model_open = tf.keras.models.load_model('{}_model_open'.format(stock))
    model_close = tf.keras.models.load_model('{}_model'.format(stock))
    
    Xtest, y_normaliser_open, y_normaliser_close = getXtest(data, data_nor)

    open_pred = model_open.predict(Xtest)
    open_pred = np.reshape(open_pred, (open_pred.shape[0], 1))
    open_pred = y_normaliser_open.inverse_transform(open_pred)
    
    close_pred = model_close.predict(Xtest)
    close_pred =np.reshape(close_pred, (close_pred.shape[0], 1))
    close_pred = y_normaliser_close.inverse_transform(close_pred)
    
    # return open_pred, close_pred
    return np.reshape(open_pred, open_pred.shape[0]), np.reshape(close_pred, close_pred.shape[0])


In [9]:
def getResult_days(stock, prediction_days):

    data = stock_dic[stock]
    data_nor = df1_dic[stock]
    
    data1 = data.to_numpy()
    
    data_normaliser = preprocessing.MinMaxScaler()
    data_normalised = data_normaliser.fit_transform(data1)

    model = tf.keras.models.load_model('{}_model_'.format(stock)+str(prediction_days)+'days')
    
    Xtest, y_normaliser_open, y_normaliser = getXtest(data, data_nor)
    
    y_test_predicted = model.predict(Xtest)
    y_test_predicted = np.reshape(y_test_predicted, (y_test_predicted.shape[0], prediction_days))
    y_test_predicted = y_normaliser.inverse_transform(y_test_predicted)
    
    y_test_prediction = []
    for i in y_test_predicted[:-1]:
        y_test_prediction.append(i[0])
    
    y_pred = y_test_prediction + list(y_test_predicted[-1])
    
    if prediction_days != 2:
        return y_pred
    
    else: # add PMA, MA if prediction_days = 2
        one_day_data = []
        day4_data = []
        ma = []

        period = len(data)-history_points
        for i in range(period-1,-1,-1):
            ma.append(sum(data['Close'].iloc[-i-7:-i-1])/6)
            one_day_data.append(data_normalised[-i-51:-i-1])
            day4_data.append(sum(data['Close'].iloc[-i-5:-i-1]))
        one_day_data = np.array(one_day_data)

        one_day_predict = model.predict(one_day_data)
        one_day_predict =np.reshape(one_day_predict, (one_day_predict.shape[0], prediction_days))
        one_day_predicted = y_normaliser.inverse_transform(one_day_predict)

        day6_sum = []
        for i in range(period):
            s = day4_data[i] + one_day_predicted[i][0] + one_day_predicted[i][1]
            day6_sum.append(s/6)

        PMA = [day6_sum[i] for i in range(period)]
        MA = [ma[i] for i in range(period)]

        UpDown = []
        for i in range(len(PMA)):
            if PMA[i] >= MA[i]:
                UpDown.append(1)
            else:
                UpDown.append(0)

        return y_pred, PMA, MA, UpDown

In [10]:
df_dic = dict()

for stock in stock_list:
    result_frame = stock_dic[stock].copy().loc[stock_dic[stock].index[history_points]:]
    idx = pd.date_range(result_frame.index[-1], periods=4, freq='B')[1:]
    open_pred, close_pred = getResult1(stock)

    pred2, PMA, MA, UpDown = getResult_days(stock, 2)
    pred3 = getResult_days(stock, 3)
#     pred30 = getResult_days(stock, 30)


    for i, date in enumerate(idx):
    # Open High Low Close Volume
#         if i ==0:
#             openpp = open_pred[-1]
#             closepp = close_pred[-1]
#             result_frame.loc[pd.to_datetime(date)] = [open_pred[-1], max(openpp,closepp), min(openpp,closepp), close_pred[-1], None]
#         else:
        result_frame.loc[pd.to_datetime(date)] = [None, None, None, None, None]
    
    
    result_frame['Open_Pred'] = list(open_pred) + [None for i in range(len(result_frame)-len(open_pred))]
    result_frame['Close_Pred1'] = list(close_pred) + [None for i in range(len(result_frame)-len(close_pred))]
    result_frame['Close_Pred2'] = list(pred2) + [None for i in range(len(result_frame)-len(pred2))]
    result_frame['Close_Pred3'] = list(pred3) + [None for i in range(len(result_frame)-len(pred3))]
#     result_frame['Close_Pred30'] = pred30
    result_frame['PMA'] = PMA + [None for i in range(len(result_frame)-len(PMA))]
    result_frame['MA'] = MA + [None for i in range(len(result_frame)-len(MA))]
    result_frame['UpDown'] = UpDown + [None for i in range(len(result_frame)-len(UpDown))]

    
    close = stock_dic[stock].Close[history_points-1:]

    colors = list()
    for i in range(1, len(close)):
        if close[i-1] <= close[i]:
            colors.append('green')
        else:
            colors.append('red')        
    
    result_frame['colors'] = colors + ['red' for i in range(len(result_frame)-len(colors))]    
    
    df_dic[stock] = result_frame

In [11]:
PIndex_MA_1 = {}
PIndex_MA_2 = {}

In [12]:
# Pindex
# 3 days prediction: close prediction (for candle stick)
# 1 day close prediction 
for key in stock_list:
    data = stock_dic[key]
    data_nor = df1_dic[key]
    
    model_close = tf.keras.models.load_model('{}_model'.format(key))
    
    Xtest, y_normaliser_open, y_normaliser_close = getXtest(data, data_nor)

    
    close_pred = model_close.predict(Xtest)
    close_pred = np.array([i[0] for i in close_pred])
    close_pred = y_normaliser_close.inverse_transform(close_pred)
    
    period = len(data)-history_points
    
    one_year = data.iloc[-period:]
    data1 = data.to_numpy()
    data_normaliser = preprocessing.MinMaxScaler()
    data_normalised = data_normaliser.fit_transform(data1)
    
    one_day_data = []
    day1_data = []
    ma = []
    for i in range(period-1,-1,-1):
        ma.append(sum(data['Close'].iloc[-i-4:-i-1])/3)
        one_day_data.append(data_normalised[-i-51:-i-1])
        day1_data.append(data['Close'].iloc[-i-1])
    one_day_data = np.array(one_day_data)
    one_day_predict = model_close.predict(one_day_data)
    one_day_predict = np.array([i[0] for i in one_day_predict])

    one_day_predicted = y_normaliser_close.inverse_transform(one_day_predict)
    
    day3_sum1 = []
    day3_sum2 = []
    for i in range(period-1):
        s = day1_data[i] +one_day_predicted[i][0] + one_day_predicted[i+1][0]

        day3_sum1.append(s/3)
 
    for i in range(period-2):
        s2 = one_day_predicted[i][0] + one_day_predicted[i+1][0]+ one_day_predicted[i+2][0]

        day3_sum2.append(s2/3)
    moving_average1 = pd.DataFrame(index = one_year.index[:-1],columns=['PIndex','MA'],data=[[day3_sum1[i],ma[i]] for i in range(period-1)])
    list1 = []
    for i in moving_average1.index:
        if moving_average1['PIndex'][i] >= moving_average1['MA'][i]:
            list1.append('green')
        else:
            list1.append('red')
    moving_average1['Trend'] = list1
    PIndex_MA_1[key] = moving_average1
    
#   Pindex 2
    moving_average2 = pd.DataFrame(index = one_year.index[:-2],columns=['PIndex','MA'],data=[[day3_sum2[i],ma[i]] for i in range(period-2)])
    list2 = []
    for i in moving_average2.index:
        if moving_average2['PIndex'][i] >= moving_average2['MA'][i]:
            list2.append('green')
        else:
            list2.append('red')
    moving_average2['Trend'] = list2
    
    PIndex_MA_2[key] = moving_average2
    
# #     model_open = tf.keras.models.load_model('{}_model_open'.format(stock))
#     model_close = tf.keras.models.load_model('{}_model'.format(key))
    
#     Xtest, y_normaliser_open, y_normaliser_close = getXtest(data, data_nor)

# # #     open_pred = model_open.predict(Xtest)
# # #     open_pred = y_normaliser_open.inverse_transform(open_pred)
    
#     close_pred = model_close.predict(Xtest)
#     close_pred = np.array([i[0] for i in close_pred]) #np.reshape(close_pred, (close_pred.shape[0], 3))
#     close_pred = y_normaliser_close.inverse_transform(close_pred)
    
#     period = len(data)-history_points
    
#     one_year = data.iloc[-period:] # one_month = data.iloc[history_points:]
#     data1 = data.to_numpy()
#     data_normaliser = preprocessing.MinMaxScaler()
#     data_normalised = data_normaliser.fit_transform(data1)
    
#     one_day_data = []
#     day1_data = []
#     ma = []
#     for i in range(period-1,-1,-1):
#         ma.append(sum(data['Close'].iloc[-i-4:-i-1])/3)
#         one_day_data.append(data_normalised[-i-51:-i-1])
#         day1_data.append(data['Close'].iloc[-i-1])
#     # for i in range(31,-1,-1):
#     #     one_day_data.append(data_normalised[-i-51:-i-1])
#     one_day_data = np.array(one_day_data)
#     one_day_predict = model_close.predict(one_day_data)
#     one_day_predict = np.array([i[0] for i in one_day_predict])
#     #one_day_predict = np.reshape(one_day_predict, (one_day_predict.shape[0], 3))
    
#     one_day_predicted = y_normaliser_close.inverse_transform(one_day_predict)
    
#     day3_sum1 = []
#     day3_sum2 = []
#     for i in range(period-1):
#         s = day1_data[i] +one_day_predicted[i][0] + one_day_predicted[i+1][0]

#         day3_sum1.append(s/3)
 
#     for i in range(period-2):
#         s2 = one_day_predicted[i][0] + one_day_predicted[i+1][0]+ one_day_predicted[i+2][0]

#         day3_sum2.append(s2/3)
#     moving_average1 = pd.DataFrame(index = one_year.index[:-1],columns=['PIndex','MA'],data=[[day3_sum1[i],ma[i]] for i in range(period-1)])
#     list1 = []
#     for i in moving_average1.index:
#         if moving_average1['PIndex'][i] >= moving_average1['MA'][i]:
#             list1.append('green')
#         else:
#             list1.append('red')
#     moving_average1['Trend'] = list1
#     PIndex_MA_1[key] = moving_average1
    
# #   Pindex 2
#     moving_average2 = pd.DataFrame(index = one_year.index[:-2],columns=['PIndex','MA'],data=[[day3_sum2[i],ma[i]] for i in range(period-2)])
#     list2 = []
#     for i in moving_average2.index:
#         if moving_average2['PIndex'][i] >= moving_average2['MA'][i]:
#             list2.append('green')
#         else:
#             list2.append('red')
#     moving_average2['Trend'] = list2
 

#     PIndex_MA_2[key] = moving_average2
    
    


In [13]:
df_dic2 = dict()

for stock in stock_list:
    result_frame = df_dic[stock]
    
    list1 = list(PIndex_MA_1[stock].Trend)
    
    constant1 = [1 for i in range(len(list1))]+ [None, None, None, None]
    PIndex_color1 = list1 + ['red', 'red', 'red', 'red']

    
    result_frame['constant1'] = constant1
    result_frame['PIndex_color1'] = PIndex_color1
    
    # pindex2
    list2 = list(PIndex_MA_2[stock].Trend)
        
    constant2 = [1 for i in range(len(list2))]+ [None, None, None, None, None]
    PIndex_color2 = list2 + ['red', 'red', 'red', 'red', 'red']

    
    result_frame['constant2'] = constant2
    result_frame['PIndex_color2'] = PIndex_color2
    

    df_dic2[stock] = result_frame

In [14]:
today = df_dic2['FB'].index[-4]
# next_date30 = df_dic['FB'].index[-1]
next_days = pd.date_range(today, periods=4, freq='B')
next_date = next_days[1]
next_date3 = next_days[-1]
# daysBTW = int(str(next_date3-today)[:2])+1

print(today)
print(next_date)
print(next_date3)

2021-12-17 00:00:00
2021-12-20 00:00:00
2021-12-22 00:00:00


In [15]:
candle_dic = dict()
for stock in stock_list:
    Open = list()
    Close = list()
    High = list()
    Low = list()
    result_frame = df_dic[stock].copy()
    for date in result_frame.index:
        if date != next_date:
            Open.append(result_frame.loc[date].Open)
            Close.append(result_frame.loc[date].Close)
            High.append(result_frame.loc[date].High)
            Low.append(result_frame.loc[date].Low)
        else:
            Open.append(result_frame.loc[next_date].Open_Pred)
            Close.append(result_frame.loc[next_date].Close_Pred1)
            High.append(max(result_frame.loc[next_date].Open_Pred,result_frame.loc[next_date].Close_Pred1))
            Low.append(min(result_frame.loc[next_date].Open_Pred,result_frame.loc[next_date].Close_Pred1))
    df = pd.DataFrame(data={'Open': Open, "High":High,"Low":Low,'Close':Close})
    df = df.set_index(result_frame.index)
    candle_dic[stock] = df

In [16]:
import pickle
f1 = open("test1.pkl","wb")
f2 = open("test2.pkl","wb")
pickle.dump(df_dic2,f1)

f1.close()
pickle.dump(candle_dic,f2)

f2.close()

Get result files and then visualization

In [17]:
options_list = [
    {'label': 'Apple', 'value': 'AAPL'},
    {'label': 'Amazon', 'value': 'AMZN'},
    {'label': 'Facebook', 'value': 'FB'},
    {'label': 'Google', 'value': 'GOOG'},
    {'label': 'Microsoft', 'value': 'MSFT'},
    {'label': 'Neflix', 'value': 'NFLX'},
    {'label': 'XLK', 'value': 'XLK'}, 
    {'label': 'QQQ', 'value': 'QQQ'}    
]

In [18]:
date_options_list = [
    {'label': '2 Days', 'value': -5},
    {'label': '5 Days', 'value': -8},
    {'label': '1 Month', 'value': -27},
    {'label': '3 Month', 'value': -69},
    {'label': '6 Month', 'value': -133},
    {'label': 'All', 'value': 'all'}  
]

In [19]:
# Initialize the app
app = dash.Dash(__name__) ## local
#app = JupyterDash(__name__) # colab
app.config.suppress_callback_exceptions = True


In [20]:
app.layout = html.Div([
# Setting the main title of the Dashboard
    html.H1("Stock Price Prediction", style={"textAlign": "center"}),
    
    html.Div([
        html.H1("Please select stock and time period.", 
            style={'textAlign': 'center'}),
            # Adding the first dropdown menu and the subsequent time-series graph
            dcc.Dropdown(id='stock_select',
                options=options_list, # multi=True,
                value='AAPL',
                style={"display": "block", "margin-left": "auto", 
                    "margin-right": "auto", "width": "60%"}),
        
            dcc.Dropdown(id='date_select',
                options=date_options_list, # multi=True,
                value=-69,
                style={"display": "block", "margin-left": "auto", 
                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='basicGraph'),                
#                 dcc.Graph(id='pindexGraph1'),
                dcc.Graph(id='pindexGraph2'),
                dcc.Graph(id='predictionGraph'),
                dcc.Graph(id='volumeGraph'),
        
            ])
])

In [21]:
@app.callback(Output('basicGraph', 'figure'),
              [Input('stock_select', 'value'), Input('date_select', 'value')])

def update_graph(selected_dropdown, selected_date):

    trace1 = []
    trace2 = []
    trace3 = []
    trace4 = []
    trace5 = []
    trace6 = []
    trace7 = []
    trace8 = []
    
    if selected_date == 'all':
        result_frame = df_dic2[selected_dropdown].copy()
        candle_frame = candle_dic[selected_dropdown].copy()
    else:
        result_frame = df_dic2[selected_dropdown].copy().iloc[selected_date:]
        candle_frame = candle_dic[selected_dropdown].copy().iloc[selected_date:]

    trace1.append(
      go.Candlestick(x=candle_frame.index, visible='legendonly',
                     open=candle_frame.Open,
                     high=candle_frame.High,
                     low=candle_frame.Low,
                     close=candle_frame.Close,
                     name=f'Candlestick'))

    trace2.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.MA,
                 mode='lines', opacity=0.8, #fill="tonexty",
                 name=f'MA', #,textposition='bottom center',
                line = dict(color='chocolate')))
    
    trace3.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.PMA,
                 mode='lines', opacity=0.9, #fill='tonexty',
                 name=f'PMA',#textposition='bottom center',
                 line = dict(color='blue')))
    
    trace4.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.Open, visible='legendonly',
                 mode='lines', opacity=0.8, line=dict(color="orange", width=4, dash='dot'),
                 name=f'Open')) #,textposition='bottom center'))
    
    trace5.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.Close, visible='legendonly',
                 mode='lines', opacity=0.6, line=dict(color="teal", width=4, dash='dash'),
                 name=f'Close')) #,textposition='bottom center'))
    
    trace6.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.Close_Pred1, visible='legendonly',
                 mode='lines', opacity=0.8, line_color= "cyan",
                 name=f'Predict 1day')) #,textposition='bottom center'))
    trace7.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.Close_Pred2, visible='legendonly',
                 mode='lines', opacity=0.8, line_color= "magenta",
                 name=f'Predict 2days')) #,textposition='bottom center'))
    
    trace8.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.Close_Pred3, visible='legendonly',
                 mode='lines', opacity=0.8, line_color= "navy",
                 name=f'Predict 3days')) #,textposition='bottom center'))
     
    traces = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8]
 
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout( #colorway=["#5E0DAC", '#800000', '#FFA500', 
                                    #        '#00FFFF', '#FF00FF','#0000FF'],
            hovermode='x unified',
            height=500, 
            margin=dict(
#             t=10, # top margin: 30px, you want to leave around 30 pixels to
#               # display the modebar above the graph.
            b=20, # bottom margin: 10px
#             l=40, # left margin: 10px
#             r=40, # right margin: 10px
            ),
            legend=dict(
                orientation="h",              
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
             title=f"{selected_dropdown}", xaxis={'rangeslider': {'visible': False}, 'type': 'date'},
#             yaxis =  {"title":"Price", "range":[min(result_frame.Open)*0.75,max(result_frame.Open*1.1)],
#                       'fixedrange': False},               
      
                  
#             yaxis2={"title":"Volume", "side":"right", "overlaying":"y", 
#                     "range":[min(result_frame.Volume),max(result_frame.Volume)*4]},                                                  
                  
                  
             shapes=[
                    dict(
                        type="rect",
                        xref="x",
                        yref="paper",
                        x0=next_date+pd.DateOffset(-1),
                        y0="0",
                        x1=result_frame.index[-1],
                        y1="1",
                        fillcolor="lightgray",
                        opacity=0.4,
                        line_width=0,
                        layer="below"
                    ),
                    ],  
                  
# #             xaxis={#"title":"Date",
# # #                    'rangeselector': {'buttons': list([
# # #                        {'count': daysBTW, 'label': '1D', 'step': 'day','stepmode': 'backward'},
# # #                        {'count': daysBTW+6, 'label': '5D', 'step': 'day', 'stepmode': 'backward'},                       
# # #                        {'count': daysBTW+30, 'label': '1M', 'step': 'day', 'stepmode': 'backward'},
# # #                        {'count': daysBTW+92, 'label': '3M', 'step': 'day', 'stepmode': 'backward'},
# # #                        {'count': daysBTW+183, 'label': '6M', 'step': 'day','stepmode': 'backward'},
# # #                        {'step': 'all'}])},                
# #                    'rangeslider': {'visible': False}, 
# #                    'type': 'date'}, 
                  

              )}    
    
    return figure

In [22]:
@app.callback(Output('pindexGraph2', 'figure'),
              [Input('stock_select', 'value'), Input('date_select', 'value')])

def update_graph4(selected_dropdown, selected_date):

    trace21 = []
    
    if selected_date == 'all':
        result_frame = df_dic2[selected_dropdown].copy()
    else:
        result_frame = df_dic2[selected_dropdown].copy().iloc[selected_date:]

   
    trace21.append(
      go.Bar(x=result_frame.index, y=result_frame.constant2,opacity=0.7, 
                      name=f'Pindex2',marker_color=result_frame.PIndex_color2)) 

  
    traces = [trace21]
 
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout( #colorway=["#5E0DAC", '#800000', '#FFA500', 
                                    #        '#00FFFF', '#FF00FF','#0000FF'],
            height=10, 
            margin=dict(
            t=0, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
            b=0, # bottom margin: 10px
#             l=40, # left margin: 10px
#             r=40, # right margin: 10px
            ),
            legend=dict(
                orientation="h",              
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
             title=f"{selected_dropdown} Pindex2", 
             xaxis={'rangeslider': {'visible': False}, 'type': 'date'},
             yaxis = {'showticklabels': False, 'showgrid': False},   
#             yaxis =  {"title":"Price", "range":[min(result_frame[:-30].Open)*0.75,max(result_frame[:-30].Open*1.1)],
#                       'fixedrange': False},
                
                  shapes=[
                    dict(
                        type="rect",
                        xref="x",
                        yref="paper",
                        x0=next_date+pd.DateOffset(-1),
                        y0="0",
                        x1=result_frame.index[-1],
                        y1="1",
                        fillcolor="lightgray",
                        opacity=0.4,
                        line_width=0,
                        layer="below"
                    ),
                    ],            
                  
#             yaxis2={"title":"Volume", "side":"right", "overlaying":"y", 
#                     "range":[min(result_frame[:-30].Volume),max(result_frame[:-30].Volume)*4]},
              )}    
                  
   
    return figure
        
        
        

In [23]:
@app.callback(Output('predictionGraph', 'figure'),
              [Input('stock_select', 'value'), Input('date_select', 'value')])
def update_graph3(selected_dropdown, selected_date):

    trace6 = []
    trace7 = []
    trace8 = []
    
    trace11 = []    
    
    if selected_date == 'all':
        result_frame = df_dic2[selected_dropdown].copy()
    else:
        result_frame = df_dic2[selected_dropdown].copy().iloc[selected_date:]


    trace6.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.Close_Pred1, #visible='legendonly',
                 mode='lines', opacity=0.8, line_color= "cyan",
                 name=f'Predict 1day')) #,textposition='bottom center'))
    trace7.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.Close_Pred2, #visible='legendonly',
                 mode='lines', opacity=0.8, line_color= "magenta",
                 name=f'Predict 2days')) #,textposition='bottom center'))
    
    trace8.append(
      go.Scatter(x=result_frame.index,
                 y=result_frame.Close_Pred3, #visible='legendonly',
                 mode='lines', opacity=0.8, line_color= "navy",
                 name=f'Predict 3days')) #,textposition='bottom center'))
    
    # next day candle: 'Open_Pred', 'Close_Pred1',
    Open = list()
    Close = list()
    High = list()
    Low = list()
    
    hovertext = list()
    
    for date in result_frame.index:
        if date != next_date:
            Open.append(None)
            Close.append(None)
            High.append(None)
            Low.append(None)
            hovertext.append(None)
        else:
            Open.append(result_frame.loc[next_date].Open_Pred)
            Close.append(result_frame.loc[next_date].Close_Pred1)
            High.append(max(result_frame.loc[next_date].Open_Pred,result_frame.loc[next_date].Close_Pred1))
            Low.append(min(result_frame.loc[next_date].Open_Pred,result_frame.loc[next_date].Close_Pred1))
            hovertext.append('Open: '+str(result_frame.loc[next_date].Open_Pred)+'<br>Close: '+str(result_frame.loc[next_date].Close_Pred1))
    
    trace11.append(
      go.Candlestick(x=result_frame.index, visible='legendonly',
                     open=Open,
                     high=High,
                     low=Low,
                     close=Close, opacity=0.7, 
                     #increasing_line_color= 'green', decreasing_line_color= 'red',
                     name=f'next candle',  
                     text = hovertext,
                     hoverinfo='text'))  
  
    traces = [trace6, trace7, trace8, trace11]
 
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout( #colorway=["#5E0DAC", '#800000', '#FFA500', 
                                    #        '#00FFFF', '#FF00FF','#0000FF'],
            hovermode='x unified',
            height=300, 
                  
            margin=dict(
            t=50, # top margin: 30px, you want to leave around 30 pixels to
#               # display the modebar above the graph.
            b=0, # bottom margin: 10px
#             l=40, # left margin: 10px
#             r=40, # right margin: 10px
            ),
            legend=dict(
                orientation="h",              
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
             #title=f"{selected_dropdown}", 
             xaxis={'rangeslider': {'visible': False}, 'type': 'date'},
                  
#             yaxis =  {"title":"Price", "range":[min(result_frame[:-30].Open)*0.75,max(result_frame[:-30].Open*1.1)],
#                       'fixedrange': False},
                
                  shapes=[
                    dict(
                        type="rect",
                        xref="x",
                        yref="paper",
                        x0=next_date+pd.DateOffset(-1),
                        y0="0",
                        x1=result_frame.index[-1],
                        y1="1",
                        fillcolor="lightgray",
                        opacity=0.4,
                        line_width=0,
                        layer="below"
                    ),
                    ],            
                  
#             yaxis2={"title":"Volume", "side":"right", "overlaying":"y", 
#                     "range":[min(result_frame[:-30].Volume),max(result_frame[:-30].Volume)*4]},
              )}    
                  
   
    return figure

In [24]:
@app.callback(Output('volumeGraph', 'figure'),
              [Input('stock_select', 'value'), Input('date_select', 'value')])

def update_graph2(selected_dropdown, selected_date):

    trace10 = []

    
    if selected_date == 'all':
        result_frame = df_dic2[selected_dropdown].copy()
    else:
        result_frame = df_dic2[selected_dropdown].copy().iloc[selected_date:]


   
    trace10.append(
      go.Bar(x=result_frame.index, y=result_frame.Volume,opacity=0.7, 
                      name=f'Volume',marker_color=result_frame.colors)) 

  
    traces = [trace10]
 
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout( #colorway=["#5E0DAC", '#800000', '#FFA500', 
                                    #        '#00FFFF', '#FF00FF','#0000FF'],
            height=300, 
            margin=dict(
            t=50, # top margin: 30px, you want to leave around 30 pixels to
#               # display the modebar above the graph.
#             b=0, # bottom margin: 10px
#             l=40, # left margin: 10px
#             r=40, # right margin: 10px
            ),
            legend=dict(
                orientation="h",              
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
              
             xaxis={'rangeslider': {'visible': False}, 'type': 'date', "title": f"{selected_dropdown} Volume"},
                  
#             yaxis =  {"title":"Price", "range":[min(result_frame[:-30].Open)*0.75,max(result_frame[:-30].Open*1.1)],
#                       'fixedrange': False},
                
                  shapes=[
                    dict(
                        type="rect",
                        xref="x",
                        yref="paper",
                        x0=next_date+pd.DateOffset(-1),
                        y0="0",
                        x1=result_frame.index[-1],
                        y1="1",
                        fillcolor="lightgray",
                        opacity=0.4,
                        line_width=0,
                        layer="below"
                    ),
                    ],            
                  
#             yaxis2={"title":"Volume", "side":"right", "overlaying":"y", 
#                     "range":[min(result_frame[:-30].Volume),max(result_frame[:-30].Volume)*4]},
              )}    
                  
   
    return figure
        
        
        

In [ ]:
# Run the app

#app.run_server(mode='inline')  # colab
if __name__ == '__main__':
    app.run_server(port = 8050)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2021 20:58:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 20:58:38] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1639889173.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 20:58:38] "GET /assets/style.css?m=1597001100.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 20:58:38] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1639889173.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 20:58:38] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1639889173.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 20:58:38] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1639889173.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 20:58:38] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1639889172.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 20:58:38] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m163988